In [ ]:
import io
import base64
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
#from loguru import logger
import numpy as np
import time
import pickle
import h5py
#from skimage.transform import downscale_local_mean,resize
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score,calinski_harabasz_score,davies_bouldin_score
import seaborn as sns
from sklearn.cluster import KMeans,DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import euclidean_distances,pairwise_distances,pairwise_distances_argmin_min
import umap
import scipy
import time
import colorcet as cc

from dash import Dash, dcc, html, Input, Output, no_update, callback, jupyter_dash
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, html
import dash_vtk
from dash_vtk.utils import to_volume_state,to_mesh_state
import vtk
import dash_bootstrap_components as dbc

from PIL import Image

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np

import itk
from itkwidgets import view
import SimpleITK as sitk

from utilities import plot_all_slices_notzero

In [ ]:
st=time.time()
with h5py.File("dataset_scapula_labelmap_res.hdf5","r") as f:
    scapula_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")

In [ ]:
scapula_dataset.shape

In [ ]:
with open("processing/scapula_res_features_250ep.pkl","rb") as f:
    features = pickle.load(f)

In [ ]:
with open("processing/scapula_res_flip_features_250ep.pkl","rb") as f:
    features = pickle.load(f)

In [ ]:
features.shape

# Calcolo array dataset-CT

In [ ]:
with open("scapula_preprocessing/arrays/pos_array_hor_all.pkl","rb") as f1:
    pos_array_hor_all = pickle.load(f1)

In [ ]:
# Devo far corrispondere ad ogni elemento del dataset la sua CT originale
dataset_el_to_ct = [] # In corrispondenza dell'indice l'elemento conterrà la CT corrispondente
for i,el in enumerate(pos_array_hor_all,start=1):
    if len(el) != 0:
        dataset_el_to_ct.append(i)
    if len(el) > 2:
        dataset_el_to_ct.append(i)

### Differenze Datset-CT (tesi)

In [ ]:
dataset_el = 0

In [ ]:
left_humerus_label = 69
right_humerus_label = 70
left_scapula_label = 71
right_scapula_label = 72
shoulder_seg_data = np.asarray(nib.load(f"processing/{ct}/shoulder_seg.nii").dataobj)
shoulder_seg_humerus_data = np.where((shoulder_seg_data == left_humerus_label) | (shoulder_seg_data == right_humerus_label),1,0).astype(np.uint8)
shoulder_seg_scapula_data = np.where((shoulder_seg_data == left_scapula_label) | (shoulder_seg_data == right_scapula_label),1,0).astype(np.uint8)

# Clustering e visualizzazione

In [ ]:
# number_clusters = 10
number_clusters = 15
kmeans = KMeans(n_clusters=number_clusters,n_init="auto",random_state=42)
labels = kmeans.fit_predict(features)

## Metriche sul clustering finale

In [ ]:
centroids = kmeans.cluster_centers_

In [ ]:
# Distanze dei centroidi (intra cluster)
centroid_distances = pairwise_distances(centroids,metric="euclidean")
plt.imshow(centroid_distances)
plt.title("Matrice delle distanze dei centroidi nelle Scapole")
plt.xticks(unique_labels)
plt.yticks(unique_labels)
plt.colorbar()

In [ ]:
# Cardinalità cluster (inter cluster)
unique_labels,cardinalities = np.unique(labels,return_counts=True)
cardinalities
plt.bar(x=unique_labels,height=cardinalities)
plt.title("Cardinalità dei Cluster nelle Scapole")
plt.xticks(unique_labels)
plt.xlabel("Cluster")
plt.ylabel("Cardinalità")

In [ ]:
# Magnitudine(media della somma delle distanze di ogni esempio dal suo centroide) (inter cluster)
dists = pairwise_distances(features,centroids)
dist_features_centroids = {el:[] for el in unique_labels}
for dist,lab in zip(dists,labels):
    dist_features_centroids[lab].append(dist[lab])
mean_dists = [np.mean(dist_features_centroids[i]) for i in unique_labels]                                        

In [ ]:
plt.bar(x=unique_labels,height=mean_dists)
plt.title("Distanza Media degli Elementi dal Centroide nelle Scapole")
plt.xticks(unique_labels)
plt.xlabel("Cluster")
plt.ylabel("Distanza media dal centroide")

## Visualizzazione

In [ ]:
# TSNE direttamente sulle feature senza PCA
st = time.time()
features_tsne = TSNE(perplexity=30,random_state=42).fit_transform(features) # Per la visualizzazione in 2D
print(f"Tempo dimensionality reduction TSNE: {time.time()-st:.2f} sec")
print(features_tsne.shape)

In [ ]:
plt.figure(figsize=(15,10))
plt.title("TSNE Scapole",fontsize=20)
sns.scatterplot(
    x = features_tsne[:,0],
    y = features_tsne[:,1],
)

In [ ]:
plt.figure(figsize=(15,10))
plt.title("Clustering TSNE Scapole",fontsize=20)
sns.scatterplot(
    x = features_tsne[:,0],
    y = features_tsne[:,1],
    hue= labels,
    palette = sns.color_palette(cc.glasbey_bw, n_colors=number_clusters)
)

In [ ]:
# UMAP
st = time.time()
features_umap = umap.UMAP(n_neighbors=30,random_state=42).fit_transform(features)
# n_neighbors basso, focus sulla local structure, alto rappresenta la struttura generale ma perde i dettagli. Range 2-100
print(f"Tempo dimensionality reduction UMAP: {time.time()-st:.2f} sec")

In [ ]:
plt.figure(figsize=(15,10))
plt.title("UMAP Scapole",fontsize=20)
sns.scatterplot(
    x = features_umap[:,0],
    y = features_umap[:,1],
)

In [ ]:
plt.figure(figsize=(15,10))
plt.title("Clustering UMAP Scapole",fontsize=24)
sns.scatterplot(
    x = features_umap[:,0],
    y = features_umap[:,1],
    hue = labels,
    palette = sns.color_palette(cc.glasbey_bw, n_colors=number_clusters)
)

# Calcolo volumi

In [ ]:
# Calcolo dei volumi di ogni CT e poi normalizzo sul volume più grande
vol = np.sum(scapula_dataset,axis=(1,2,3))
maxvol = np.max(vol)
rel_vols = vol / maxvol
rel_vols.shape

In [ ]:
with open("processing/scapula_volumes.pkl","wb") as f:
    pickle.dump(rel_vols,f)

In [ ]:
with open("processing/scapula_volumes.pkl","rb") as f:
    rel_vols = pickle.load(f)
rel_vols.shape

# 4 nearest neighbors

In [ ]:
# Prendo i 4 punti vicini ad ogni centroide. Sono in ordine di distanza, il primo è il più vicino
neighbors = NearestNeighbors(n_neighbors=4).fit(features)
distances,indexes = neighbors.kneighbors(kmeans.cluster_centers_)

In [ ]:
indexes

In [ ]:
# Creo il corrispondente array ma con i numeri delle CT al posto degli indici del dataset
indexes_ct = []
for cluster in indexes:
    cluster_ct = []
    for el in cluster:
        cluster_ct.append(dataset_el_to_ct[el])
    indexes_ct.append(cluster_ct)
indexes_ct

In [ ]:
view(scapula_dataset[indexes[0][0]]*255)

In [ ]:
view(scapula_dataset[indexes[0][2]]*255)

# Coregistrazione

In [ ]:
# Consigli trovati sul manuale
parameter_object = itk.ParameterObject.New()
parameter_map_custom = parameter_object.GetDefaultParameterMap('affine')
# parameter_map_custom['Transform'] = ['SimilarityTransform'] # Solo traslaz, rotaz e scaling isotropico
parameter_map_custom['Metric'] = ['AdvancedKappaStatistic'] # Specifico per binary images
parameter_map_custom['FinalBSplineInterpolationOrder'] = ['0'] # Per produrre una binary image
parameter_object.AddParameterMap(parameter_map_custom)

In [ ]:
st = time.time()
fixed_im = scapula_dataset[indexes[0][0]] # Più vicino al cluster 0
moving_im = scapula_dataset[indexes[1][0]] # Più vicino al cluster 1 
registered_image, params = itk.elastix_registration_method(fixed_im,moving_im,parameter_object=parameter_object)
print(f"Tempo: {time.time()-st:.2f} sec")

In [ ]:
# Visualizzazione 3D 
# view(fixed_im-registered_image_labelmap,label_image=fixed_im)
view(fixed_im-registered_image)

# Dash APP Clustering

In [ ]:
use_tsne = False # True uso TSNE, False uso UMAP

In [ ]:
# Per produrre l'immagine 
def np_image_to_base64(scapula_dataset,slice_x,slice_y,slice_z):
    im_x = Image.fromarray(scapula_dataset[slice_x,:,:]*255)
    im_y = Image.fromarray(np.pad(scapula_dataset[:,slice_y,:],((15,15),(0,0)))*255)
    im_z = Image.fromarray(np.pad(scapula_dataset[:,:,slice_z],((15,15),(57,56)))*255)
    
    img_conc = Image.new("L",(504,391*3))
    img_conc.paste(im_x,(0,0))
    img_conc.paste(im_y,(0,391))
    img_conc.paste(im_z,(0,391*2))
    
    buffer = io.BytesIO()
    img_conc.save(buffer, format="jpeg")
    encoded_image = base64.b64encode(buffer.getvalue()).decode()
    im_url = "data:image/jpeg;base64, " + encoded_image
    return im_url

# DASH app
app = Dash(__name__)

app.layout = html.Div(
    className="container",
    children=[
        html.Div([
            html.P("Modalità di visualizzazione:"),
            dcc.RadioItems(
                id='graph-info', 
                value='Clusters', 
                options=['Clusters', 'Volumes'],
                inline=True,
            )
        ],
        style={"margin-left":"5em"}),
        # dcc.Graph(id="graph-5", figure=fig, clear_on_unhover=True),
        dcc.Graph(id="graph-5", clear_on_unhover=True),
        dcc.Tooltip(id="graph-tooltip-5", direction='left'),
    ],
)

@app.callback(
    Output("graph-tooltip-5", "show"),
    Output("graph-tooltip-5", "bbox"),
    Output("graph-tooltip-5", "children"),
    Output("graph-5", "figure"),
    Input("graph-5", "hoverData"),
    Input("graph-info", "value"))
def display_hover(hoverData,graph_info):
    # Figura dinamica
    fig = px.scatter(
        x=features_tsne[:,0] if use_tsne else features_umap[:,0],
        y=features_tsne[:,1] if use_tsne else features_umap[:,1],
        color=rel_vols if graph_info=="Volumes" else labels.astype(str), # Trasformo in stringhe per avere discreto
        )

    fig.update_traces(
        hoverinfo="none",
        hovertemplate=None,
    )

    fig.update_layout(
        autosize=False,
        width=1450,
        height=800
    )
    
    if hoverData is None:
        return False, no_update, no_update,fig

    hover_data = hoverData["points"][0]
    bbox = hover_data["bbox"]
    num = hover_data["pointNumber"]

    #im_matrix = scapula_dataset[num] # Qui va la CT, num è il numero del pallino su cui si passa il puntatore
    #im_url = np_image_to_base64(im_matrix,150,150,250)
    children = [
        html.Div([
            #html.Img(src=im_url,style={"width": "120px", 'display': 'block', 'margin': '0 auto'}),
            html.P("CT " + str(dataset_el_to_ct[num]), style={'font-weight': 'bold'}), # Occhio che Num non è proprio la CT
            html.P(f"Volume: {rel_vols[num]:.3f}")
        ])
    ]

    return True, bbox, children, fig


if __name__ == "__main__":
    app.run(jupyter_mode="external",host="0.0.0.0",port="8090") # Qui metto la porta tcp
    app.title = "Scapula TSNE" if use_tsne else "Scapula UMAP"

## Codice vecchio

In [ ]:
# Elenco dei colori per ogni cluster
color_map = {
    0: "#E52B50",
    1: "#9F2B68",
    2: "#3B7A57",
    3: "#3DDC84",
    4: "#FFBF00",
    5: "#915C83",
    6: "#008000",
    7: "#7FFFD4",
    8: "#E9D66B",
    9: "#007FFF",
}
colors = [color_map[l] for l in labels] # Assegno ad ogni label il suo colore

fig = go.Figure(data=[go.Scatter(
    # TSNE o UMAP
    x=features_tsne[:,0] if use_tsne else features_umap[:,0],
    y=features_tsne[:,1] if use_tsne else features_umap[:,1],
    mode='markers',
    marker=dict(
        size=6,
        color=colors,
    )
)])

fig.update_traces(
    hoverinfo="none",
    hovertemplate=None,
)

fig.update_layout(
    autosize=False,
    width=1500,
    height=800
)


# DASH app
app = Dash(__name__)

app.layout = html.Div(
    className="container",
    children=[
        dcc.Graph(id="graph-5", figure=fig, clear_on_unhover=True),
        dcc.Tooltip(id="graph-tooltip-5", direction='left'),
    ],
)

@app.callback(
    Output("graph-tooltip-5", "show"),
    Output("graph-tooltip-5", "bbox"),
    Output("graph-tooltip-5", "children"),
    Input("graph-5", "hoverData"))
def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    hover_data = hoverData["points"][0]
    bbox = hover_data["bbox"]
    num = hover_data["pointNumber"]

    im_matrix = scapula_dataset[num] # Qui va la CT, num è il numero del pallino su cui si passa il puntatore
    # Ad esempio mostro la slice 250 di ogni CT
    im_url = np_image_to_base64(im_matrix,150,150,250)
    children = [
        html.Div([
            html.Img(
                src=im_url,
                style={"width": "120px", 'display': 'block', 'margin': '0 auto'},
            ),
            html.P("CT " + str(dataset_el_to_ct[num]), style={'font-weight': 'bold'}) # Occhio che Num non è proprio la CT
        ])
    ]

    return True, bbox, children

def np_image_to_base64(im_matrix):
    # Vecchia
    im = Image.fromarray(im_matrix)
    buffer = io.BytesIO()
    im.save(buffer, format="jpeg")
    encoded_image = base64.b64encode(buffer.getvalue()).decode()
    im_url = "data:image/jpeg;base64, " + encoded_image
    return im_url

# Dash Rendering 3D

## Nearest Neighbors

In [ ]:
# Prendo i 4 punti vicini ad ogni centroide. Sono in ordine di distanza, il primo è il più vicino
centroids = kmeans.cluster_centers_
neighbors = NearestNeighbors(n_neighbors=4).fit(features)
distances,indexes = neighbors.kneighbors(centroids)

In [ ]:
distances,indexes

In [ ]:
# Creo il corrispondente array ma con i numeri delle CT al posto degli indici del dataset
indexes_ct = []
for cluster in indexes:
    cluster_ct = []
    for el in cluster:
        cluster_ct.append(dataset_el_to_ct[el])
    indexes_ct.append(cluster_ct)
indexes_ct = np.asarray(indexes_ct)
indexes_ct

In [ ]:
with open("processing/scapula_indexes_ct.pkl","rb") as f:
    indexes_ct = pickle.load(f)
    
with open("processing/scapula_indexes_ct_flip.pkl","rb") as f:
    indexes_ct_flip = pickle.load(f)

## Salvataggio Immagini in VTK

Effettuando un cropping alla bounding box minima per ridurre al massimo le dimensioni e rendere i caricamenti rapidi

In [ ]:
indexes_ct_flip

In [ ]:
# Se voglio visualizzare le scapole flippate
with open("processing/scapula_flipped_indexes.pkl","rb") as f:
    flipped_idxs = pickle.load(f)

In [ ]:
print(flipped_idxs)

In [ ]:
def bbox_3D(img):

    r = np.any(img, axis=(1, 2))
    c = np.any(img, axis=(0, 2))
    z = np.any(img, axis=(0, 1))

    rmin, rmax = np.where(r)[0][[0, -1]]
    cmin, cmax = np.where(c)[0][[0, -1]]
    zmin, zmax = np.where(z)[0][[0, -1]]

    return rmin, rmax, cmin, cmax, zmin, zmax

In [ ]:
for ct in indexes_ct_flip.flatten():
    if dataset_el_to_ct.index(ct) in flipped_idxs:
        print(ct)

In [ ]:
# Salvo tutte le immagini in formato vtk con cropping per risparmiare tempo
for ct in indexes_ct_flip.flatten():
    # ct = dataset_el_to_ct[index]
    original_shoulder = nib.load(f"processing/{ct}/shoulder_nifti_res.nii")
    segmented_shoulder = nib.load(f"processing/{ct}/shoulder_seg_res.nii")

    original_shoulder_data = np.asarray(original_shoulder.dataobj)
    segmented_shoulder_data = np.asarray(segmented_shoulder.dataobj)

    left_scapula_label = 71
    right_scapula_label = 72

    # Segmento solo la scapola dai dati originali applicando la maschera
    original_shoulder_data_segmented = np.where((segmented_shoulder_data == left_scapula_label) | (segmented_shoulder_data == right_scapula_label),original_shoulder_data,0)
    
    # Taglio
    bbox = bbox_3D(original_shoulder_data_segmented)
    original_shoulder_data_segmented_cropped = original_shoulder_data_segmented[bbox[0]:bbox[1],bbox[2]:bbox[3],bbox[4]:bbox[5]]
    
    if dataset_el_to_ct.index(ct) in flipped_idxs:
        original_shoulder_data_segmented_cropped = np.flip(original_shoulder_data_segmented_cropped,axis=0)
        sitk.WriteImage(sitk.GetImageFromArray(original_shoulder_data_segmented_cropped), f"vtk_images/scapula_{ct}_segm_flip_mirrored.vtk")
        print(f"Scapola {ct} salvata")

## Dash APP Rendering 3D (su file .py)

In [ ]:
# Immagine di esempio per evitare caricamenti lunghi
try:
    from vtkmodules.vtkImagingCore import vtkRTAnalyticSource
except ImportError:
    from vtk.vtkImagingCore import vtkRTAnalyticSource

# Use VTK to get some data
data_source = vtkRTAnalyticSource()
data_source.Update()  # <= Execute source to produce an output
dataset = data_source.GetOutput()

volume_state = to_volume_state(dataset)

content_cube = dash_vtk.View([ 
    dash_vtk.VolumeRepresentation([
        dash_vtk.VolumeController(),
        dash_vtk.Volume(state=volume_state)
    ],
    colorMapPreset='erdc_rainbow_dark'
    ),
])

In [ ]:
def create_col(title,image,wid_cols=4,heig=300):
    """
    Crea un'immagine della griglia con titolo e renering
    """
    column = dbc.Col(
                html.Div([
                    html.H4(children=title),
                    html.Div(
                    style={"width": "100%", "height": f"{heig}px"},
                    children=[image]
                    ),
                ]),
                width=wid_cols
            )
    return column

In [ ]:
def create_row(cluster,use_flipped=True,show_flip=True):
    """
    Ritorna le tre immagini più vicine al centroide del cluster passato come parametro e i rispettivi numeri di CT 
    """
    reader = vtk.vtkDataSetReader() # Devo usare questo reader
    if use_flipped:
        ct_0 = indexes_ct_flip[cluster][0]
        ct_1 = indexes_ct_flip[cluster][1]
        ct_2 = indexes_ct_flip[cluster][2]
    else:
        ct_0 = indexes_ct[cluster][0]
        ct_1 = indexes_ct[cluster][1]
        ct_2 = indexes_ct[cluster][2]
    
    if use_flipped:
        if show_flip and os.path.exists(f"vtk_images/scapula_{ct_0}_segm_flip_mirrored.vtk"):
            reader.SetFileName(f"vtk_images/scapula_{ct_0}_segm_flip_mirrored.vtk")
        else:
            reader.SetFileName(f"vtk_images/scapula_{ct_0}_segm_flip.vtk")
    else:
        reader.SetFileName(f"vtk_images/scapula_{ct_0}_segm.vtk")
    reader.Update()
    volume_state_0 = to_volume_state(reader.GetOutput())

    scapula_0 = dash_vtk.View([
        dash_vtk.VolumeRepresentation([
            dash_vtk.VolumeController(),
            dash_vtk.Volume(state=volume_state_0),
        ],
            colorMapPreset="X Ray",
            # property={"diffuse":"0.3"}
        ),
    ])
    
    if use_flipped:
        if show_flip and os.path.exists(f"vtk_images/scapula_{ct_1}_segm_flip_mirrored.vtk"):
            reader.SetFileName(f"vtk_images/scapula_{ct_1}_segm_flip_mirrored.vtk")
        else:
            reader.SetFileName(f"vtk_images/scapula_{ct_1}_segm_flip.vtk")
    else:
        reader.SetFileName(f"vtk_images/scapula_{ct_1}_segm.vtk")
    reader.Update()
    volume_state_1 = to_volume_state(reader.GetOutput())

    scapula_1 = dash_vtk.View([
        dash_vtk.VolumeRepresentation([
            dash_vtk.VolumeController(),
            dash_vtk.Volume(state=volume_state_1),
        ]),
    ])
    
    if use_flipped:
        if show_flip and os.path.exists(f"vtk_images/scapula_{ct_2}_segm_flip_mirrored.vtk"):
            reader.SetFileName(f"vtk_images/scapula_{ct_2}_segm_flip_mirrored.vtk")
        else:
            reader.SetFileName(f"vtk_images/scapula_{ct_2}_segm_flip.vtk")
    else:
        reader.SetFileName(f"vtk_images/scapula_{ct_2}_segm.vtk")
    reader.Update()
    volume_state_2 = to_volume_state(reader.GetOutput())

    scapula_2 = dash_vtk.View([
        dash_vtk.VolumeRepresentation([
            dash_vtk.VolumeController(),
            dash_vtk.Volume(state=volume_state_2),
        ]),
    ])
    
    return scapula_0,scapula_1,scapula_2,ct_0,ct_1,ct_2

In [ ]:
# Con 2 callback diversi riesco a separare i due caricamenti e aggiornare solo la riga che cambia
use_flip = True # Se True uso le flipped se false il dataset originale
show_flip = True # Se True mostro le scapole ribaltate, se False mostro quelle originali

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

dropdown_style = {
    "width":"50%",
    "margin-top":"1em",
    "margin-bottom":"1em",
}
dropdown_list = [f"Cluster {x}" for x in range(len(indexes_ct_flip))] if use_flip else [f"Cluster {x}" for x in range(len(indexes_ct))]

app.layout = dbc.Container([
    dcc.Dropdown(dropdown_list,"Cluster 0",id="dropdown_1",style=dropdown_style,clearable=False),
    dbc.Row(id="row_1"),
    dcc.Dropdown(dropdown_list,"Cluster 1",id="dropdown_2",style=dropdown_style,clearable=False),
    dbc.Row(id="row_2",style={"margin-bottom":"5em"}),
])

@app.callback(
    Output("row_1","children"),
    Input("dropdown_1","value"),
)
def update_row_1(drop_1):  
    cluster_1 = int(drop_1.split()[1])
    
    scapula_1_1,scapula_1_2,scapula_1_3,ct_1_1,ct_1_2,ct_1_3 = create_row(cluster_1,use_flipped=use_flip,show_flip=show_flip)
    row_1 = [
        create_col(f"Cluster {cluster_1} Scapola {ct_1_1}",scapula_1_1),
        create_col(f"Cluster {cluster_1} Scapola {ct_1_2}",scapula_1_2),
        create_col(f"Cluster {cluster_1} Scapola {ct_1_3}",scapula_1_3),
    ]
    
    return row_1

@app.callback(
    Output("row_2","children"),
    Input("dropdown_2","value"),
)
def update_row_2(drop_2):  
    cluster_2 = int(drop_2.split()[1])

    scapula_2_1,scapula_2_2,scapula_2_3,ct_2_1,ct_2_2,ct_2_3 = create_row(cluster_2,use_flipped=use_flip,show_flip=show_flip)
    row_2 = [
        create_col(f"Cluster {cluster_2} Scapola {ct_2_1}",scapula_2_1),
        create_col(f"Cluster {cluster_2} Scapola {ct_2_2}",scapula_2_2),
        create_col(f"Cluster {cluster_2} Scapola {ct_2_3}",scapula_2_3),
    ]
    
    return row_2

app.run(jupyter_mode="external",host="0.0.0.0",port="8090") # Qui metto la porta tcp
app.title = "Scapula 3D Rendering"

In [ ]:
indexes_ct,indexes_ct_flip